**Nguyễn Thị Kim Oanh**

**MSSV: 18020989**

**Lớp: K63K1**


In [ ]:
pip install Levenshtein

In [ ]:
#Giải nén các file dữ liệu và lưu vào output
!unzip ../input/home-depot-product-search-relevance/product_descriptions.csv.zip
!unzip ../input/home-depot-product-search-relevance/train.csv.zip
!unzip ../input/home-depot-product-search-relevance/test.csv.zip
!unzip ../input/home-depot-product-search-relevance/attributes.csv.zip

In [ ]:
#Khai báo các thư viện cần thiết
%matplotlib inline
import re
import nltk
import numpy as np 
import json
import os
import pandas as pd
import seaborn as sns # vừa đếm số lượng vừa vẽ lên biểu đồ
import Levenshtein # Đo khoảng cách giữa 2 string (độ tương đồng)
from matplotlib import pyplot as plt # vẽ biểu đồ
from matplotlib_venn import venn2 # vẽ biểu đồ nhưng là dạng tròn
from nltk.corpus import stopwords # các stopwords (the, or, and, ...)
from nltk.stem.snowball import SnowballStemmer # kỹ thuật stemmer đưa về từ gốc
from nltk.stem import WordNetLemmatizer # kỹ thuật Lemmatizer đưa về từ gốc
stemmer = SnowballStemmer('english') # stemer với ngôn ngữ là tiếng anh
nltk.download('wordnet') # dowload bộ từ điển cho Lemmatizer
nltk.download('stopwords') # dowload bộ các stopwords
stop_words = set(stopwords.words('english')) # các stopwords là tiếng anh 

In [ ]:
#đọc file dữ liệu
dt_train = pd.read_csv("train.csv", encoding="ISO-8859-1")
dt_test = pd.read_csv("test.csv", encoding="ISO-8859-1")
dt_attributes = pd.read_csv("attributes.csv")
dt_descriptions = pd.read_csv("product_descriptions.csv")

In [ ]:
#check số giá trị null trong các file dữ liệu
print(f"train data has {dt_train.isnull().values.sum()} null values:")
print(f"test data has {dt_test.isnull().values.sum()} null values:")
print(f"attributes data has {dt_attributes.isnull().values.sum()} null values:")
print(f"descriptions data has {dt_descriptions.isnull().values.sum()} null values:")

In [ ]:
# hàm show thông tin của tập data
def show_data_info(dt):
    print(f"df shape: \n",dt.shape,"\n")
    print("df columns: \n",dt.columns,"\n")
    dt.info()
    return dt.head(5)

# **Training data**
* Chứa các trường dữ liệu product_uid, product_title, search_term và relevance

In [ ]:
#show thông tin tập dt_train
show_data_info(dt_train)

In [ ]:
print("There are in total {} products ".format(len(dt_train.product_title.unique())))
print("There are in total {} search query ".format(len(dt_train.search_term.unique())))
print("There are in total {} product_uid".format(len(dt_train.product_uid.unique())))

In [ ]:
# Thống kê số lượng các rating trong tập dt_train
def count_SR():
    high_SR = [i for i in dt_train['relevance'] if i == 3.00]
    midle_SR = [i for i in dt_train['relevance'] if i > 1.00 and i < 3.00]
    low_SR = [i for i in dt_train['relevance'] if i == 1.00]
    return [len(high_SR), len(midle_SR), len(low_SR)] 

In [ ]:
plt.bar(['high_sr', 'midle_sr', 'low_sr'], count_SR(), label='Relevance score', color='green')
plt.xlabel('Ratings')
plt.ylabel('Numbers')
plt.title('Numbers of these ratings')
plt.legend()
plt.show()

In [ ]:
# chi tiết hơn về số lượng relevance
sns.countplot(x="relevance", data=dt_train)
plt.show()

# **Testing data**
* Chứa các trường dữ liệu product_uid, product_title, search_term


In [ ]:
#show thông tin tập dt_train
show_data_info(dt_test)

In [ ]:
print("There are in total {} products ".format(len(dt_test.product_title.unique())))
print("There are in total {} search query ".format(len(dt_test.search_term.unique())))
print("There are in total {} product_uid".format(len(dt_test.product_uid.unique())))

In [ ]:
#biểu đồ biểu thị số lượng của các product_uid xuất hiện ở 2 tập dt_test và dt_train
venn2([set(dt_train["product_uid"]), set(dt_test["product_uid"])], set_labels=('train', 'test'), set_colors=('green', 'blue'))
plt.show()

In [ ]:
# biểu đồ biểu thị số lượng của các search_term xuất hiện ở 2 tập dt_test và dt_train
venn2([set(dt_train["search_term"]), set(dt_test["search_term"])], set_labels=('train', 'test'), set_colors=('green', 'blue'))
plt.show()

# **Descriptions data**
* Chứa các trường product_uid và product_description
* Chứa mô tả văn bản của từng sản phẩm.


In [ ]:
#show thông tin tập dt_descriptions
show_data_info(dt_descriptions)

In [ ]:
print("There are in total {} product_uid ".format(len(dt_descriptions.product_uid.unique())))
print("There are in total {} product_descriptions ".format(len(dt_descriptions.product_description.unique())))

# **Attributes data**
* Chứa các trường product_uid, name và value
* Cung cấp thông tin mở rộng về một tập hợp con của các sản phẩm (thường đại diện cho các thông số kỹ thuật chi tiết). Không phải mọi sản phẩm sẽ có các thuộc tính.

In [ ]:
#show thông tin tập dt_attributes
show_data_info(dt_attributes)

# **Clean data**
* Biến đổi tập dt_description và dt_attributes

In [ ]:
# hàm loại bỏ duplicates
def remove_duplicates(string):
    lits_tokens = [] 
    [lits_tokens.append(str(_)) for _ in string.split() if _ not in lits_tokens]
    return ' '.join(lits_tokens)

In [ ]:
# Lấy phần chung lớn nhất trong trường name là bullet thay vì bullet0 x y z
dt_attributes['name'] = [_[:6] if 'bullet' in str(_).lower() else _ for _ in dt_attributes['name'].tolist()]

# tạo nên trường product_attributes từ name và value 
dt_attributes['product_attributes'] = dt_attributes['name'] + ' ' +  dt_attributes['value']
dt_attributes = dt_attributes.drop(['name', 'value'], axis=1)

# gộp các thuộc tính có chung id lại thành một và nối các product_attributes tương ứng lại với nhau
# sử dụng thêm hàm astype() để ép tất các tokens về string vì có chứa số trong trường value.
# để nối các giá trị sau khi nhóm thì dùng hàm aggregate(), các giá trị ở cột value có chung id sẽ được nối với nhau cách nhau một khoảng trắng
dt_attributes = dt_attributes.groupby('product_uid').aggregate({'product_attributes': lambda _ : ' '.join(_.astype(str))})

# lấy phần chung của name và gỡ các tokens bị lặp.
dt_attributes['product_attributes'] = [remove_duplicates(_) for _ in dt_attributes.product_attributes.tolist()]

In [ ]:
# Nối bảng dt_attributes với bảng dt_descriptions bằng thuộc tính chung là product_id
dt_des_attr = pd.merge(dt_descriptions, dt_attributes, on='product_uid', how='left')

In [ ]:
#show thông tin bảng vừa nối
dt_des_attr.info()

In [ ]:
# tìm và thay tất cả các giá trị Null bằng ''
dt_des_attr['product_attributes'].fillna('', inplace = True)

In [ ]:
# xóa các câu bị copy nhầm trong trường product_descriptions bao gồm các đoạn mô tả trong html, link.
strings = ['br',
           'src',
           'href',
           'alt',
           'please visit'
           'Click here to review our return policy for additional information regarding returns', 
           'Click here to see Home Depot', 
           'Click here for our Project Guide', 
           'Click here for our Buying Guide', 
           'Click on the More Info tab to download',
           'CLICK HERE to create your own collection',
           'Click Here for details on the services',
           'Click Here for Ideas and Designs',
           'Click Here for a Demo of the Design',
           'Click Here to learn more about',
           'CLICK HERE to view our',
           'Click below to visit our',
           'Click here to purchase a sample of this',
           'click on the link to get started',
           'Click image to enlarge',
           'https://www.ryobitools.com/nation',
           'http://www.homedepot.com/ApplianceDeliveryandInstallation',
           'http://itemvideo-dev.microsite.homedepot.com/111414/26P/online_BB_banner_111114.jpg',
           'http://www.homedepot.com/p/Rev-A-Shelf-Door-Mounting-Kit-5WB-DMKIT/202855698']

for string in strings:
    dt_des_attr['product_description'] = [_.lower().replace(string.lower(), '') for _ in dt_des_attr.product_description]

In [ ]:
#show 5 hàng đầu
dt_des_attr.head(5)

In [ ]:
# tạo một trường mới chứa cả thông tin mô tả sản phẩm và giá trị thuộc tính của nó.
# trường product_description_attributes = product_description + product_attributes
dt_des_attr['product_description_attributes'] = dt_des_attr['product_description'] + ' ' + dt_des_attr['product_attributes']

# xóa 2 trường cũ
dt_des_attr = dt_des_attr.drop(['product_description', 'product_attributes'], axis=1)

In [ ]:
# kết quả sau khi nối.
dt_des_attr.head(5)

In [ ]:
# nối tập dt_train với dt_des_attr
dt_train = pd.merge(dt_train, dt_des_attr, on='product_uid', how='left')

# nối tập dt_test với tập dt_des_attr
dt_test = pd.merge(dt_test, dt_des_attr, on='product_uid', how='left')

In [ ]:
#show kết quả
dt_train.head(5)

In [ ]:
#show kết quả
dt_test.head(5)

**Các hàm Clean data**

In [ ]:
#lấy dữ liệu từ file .json để sửa lỗi chính tả 
#thay các giá trị giống trường key bằng trường value
spell_check = json.load(open('../input/spell-check-dict-kimoanh29/spell_check.json', 'r'))
def speel_fix(string):
    for (k,v) in spell_check.items():
        string = string.replace(k,v)
    return string


# **Một số hàm clean data khác**

In [ ]:
def standardized_sentences(string):
    string = string.lower()

    # gỡ các ký hiệu sản phẩm không liên quan
    string = string.replace('-', '')
    string = string.replace('in.', 'inch')
    string = string.replace('ft.', 'foot')
    string = string.replace('-oz. ', ' ')
    string = string.replace('oz.', ' ')
    string = string.replace('sq.', ' ')
    string = string.replace('Gal.', ' ')
    string = string.replace('lb.', ' ')
    string = string.replace('cu.', ' ')
    string = string.replace('O.D.', ' ')
    string = string.replace('sq.', ' ')
    string = string.replace('st.', ' ')
    string = string.replace('lb.', ' ')
    string = string.replace('Dia.', ' ')
    string = string.replace('dia.', ' ')

    # Thay thế tất cả các ký tự đặc biệc bằng 1 space
    string = re.sub(r'[^a-zA-Z0-9]+', ' ', string)

    # Thay thế các ký tự đứng một mình bằng 1 space
    string = re.sub(r'\b[a-zA-Z]\b', ' ', string)

    # Gộp các space đứng liền nhau thành 1 space
    string = re.sub(r'\s+', ' ', string)
   
    return string

In [ ]:
def standardized_search_term(string):
    string = string.lower()

    # gỡ các ký hiệu sản phẩm không liên quan
    string = string.replace('in.', 'inch')
    string = string.replace('ft.', 'foot')
    string = string.replace('ft', 'foot')

    # key word bị viết riêng lẻ
    string = string.replace('r ', 'r')

    # Thay thế tất cả các ký tự đặc biệc bằng 1 space
    string = re.sub(r'[^a-zA-Z0-9]+', ' ', string)

    # Gộp các space đứng liền nhau thành 1 space
    string = re.sub(r'\s+', ' ', string)

    return string

In [ ]:
# Đưa các từ về từ gốc của nó.
# áp dụng hàm này cho cả 3 trường dữ liệu title, search_term và des_attr
def set_root_form(string):
    lemmatizer = WordNetLemmatizer()
    return ' '.join(map(lambda x: lemmatizer.lemmatize(x), list(map(lambda x: stemmer.stem(x), string.split()))))

In [ ]:
# gỡ các stopwords
# hàm này chỉ thực hiện ở trường mô tả và thuộc tính
def remove_stopwords(string):
    return ' '.join([w for w in string.split() if not w in stop_words])

# **Clean tập train**

In [ ]:
# Sửa lỗi chính tả cho các trường search_term, product_title và product_description_attributes
dt_train['search_term'] = dt_train['search_term'].map(lambda x:speel_fix(x))
dt_train['product_title'] = dt_train['product_title'].map(lambda x:speel_fix(x))
dt_train['product_description_attributes'] = dt_train['product_description_attributes'].map(lambda x:speel_fix(x))

In [ ]:
# chuẩn hóa trường search_term
dt_train['search_term'] = [standardized_search_term(_) for _ in dt_train.search_term]

In [ ]:
# chuẩn hóa trường product_title và product_description_attributes
dt_train['product_title'] = [standardized_sentences(_) for _ in dt_train.product_title]
dt_train['product_description_attributes'] = [standardized_sentences(_) for _ in dt_train.product_description_attributes]

In [ ]:
# gỡ các stopwords trong trường product_description_attributes
dt_train['product_description_attributes'] = [remove_stopwords(_) for _ in dt_train.product_description_attributes]

In [ ]:
# đưa các từ về dạng gốc của nó
dt_train['search_term'] = [set_root_form(_) for _ in dt_train.search_term] 
dt_train['product_title'] = [set_root_form(_) for _ in dt_train.product_title]
dt_train['product_description_attributes'] = [set_root_form(_) for _ in dt_train.product_description_attributes] 

In [ ]:
#show kết quả
dt_train.head(5)

# **Clean tập test**

In [ ]:
# Sửa lỗi chính tả cho các trường search_term, product_title và product_description_attributes
dt_test['search_term'] = dt_test['search_term'].map(lambda x:speel_fix(x))
dt_test['product_title'] = dt_test['product_title'].map(lambda x:speel_fix(x))
dt_test['product_description_attributes'] = dt_test['product_description_attributes'].map(lambda x:speel_fix(x))

In [ ]:
# chuẩn hóa trường search_term
dt_test['search_term'] = [standardized_search_term(_) for _ in dt_test.search_term]

In [ ]:
# chuẩn hóa trường product_title và product_description_attributes
dt_test['product_title'] = [standardized_sentences(_) for _ in dt_test.product_title]
dt_test['product_description_attributes'] = [standardized_sentences(_) for _ in dt_test.product_description_attributes]

In [ ]:
# gỡ các stopwords trong trường product_description_attributes
dt_test['product_description_attributes'] = [remove_stopwords(_) for _ in dt_test.product_description_attributes]

In [ ]:
# đưa các các từ về dạng gốc của nó
dt_test['search_term'] = [set_root_form(_) for _ in dt_test.search_term]
dt_test['product_title'] = [set_root_form(_) for _ in dt_test.product_title]
dt_test['product_description_attributes'] = [set_root_form(_) for _ in dt_test.product_description_attributes]

In [ ]:
#show kết quả
dt_test.head(5)

# **Viết hàm cho phép quan sát sự ảnh hưởng của độ dài các trường tới relevance**

In [ ]:
def correlation(dt_sample, dt_field, transform=True):
    # Lấy độ dài của trường thông tin
    # Đặt transform=True thì sau này  chúng ta có thể sử dụng transform=False để lấy chính dữ liệu ở trường đó mà không phải là độ dài :) do đó mà có thể xem được sự ảnh hưởng của chính trường dữ liệu đó lên relevance
    x_ar = np.array(dt_sample[dt_field].map(lambda x:len(str(x).split())).astype(np.int64)) if transform else dt_sample[dt_field]
   
    # Lấy relevance tương ứng
    y_ar = np.array(dt_sample['relevance'])
   
    # Vẽ các điểm (độ dài, relevance) lên đồ thị
    plt.plot(x_ar, y_ar, 'bo')
   
    # tính toán m và b để vẽ đường regression
    m, b = np.polyfit(x_ar, y_ar, 1)
   
    # Vẽ regression
    plt.plot(x_ar, m * x_ar + b,'r')

In [ ]:
# tập dt_train raw chưa bị biến đổi
dt_raw_train = pd.read_csv("./train.csv", encoding="ISO-8859-1")

# biểu diễn với trường product_title
correlation(dt_raw_train, 'product_title')

In [ ]:
# biểu diễn với trường search_term
correlation(dt_raw_train, 'search_term')

In [ ]:
# biểu diễn với trường product_description_attributes
correlation(dt_train, 'product_description_attributes')

**Viết các hàm lấy để tạo features từ độ xuất hiện của các từ trong search_term**

In [ ]:
# Tìm các tokens chung 
# tách tokens của 2 câu bằng hàm split() rồi lần lượt tính sự xuất hiện của tokens ở câu 1 trong câu 2
def str_common_tokens(sentence_1, sentence_2):
    return sum(1 for word in str(sentence_2).split() if word in set(str(sentence_1).split()))

In [ ]:
# Các từ chung một phần nào đấy
# để lấy các từ chung một phần thì ta không tách tokens (ko dùng hàm split() ở đây)
# lần lượt tính sự xuất hiện của các từ trong câu 1 ở câu 2
def str_common_word(sentence_1, sentence_2):
    return sum(1 for word in str(sentence_2) if word in set(sentence_1))

In [ ]:
# Tính tổng tất cả các Tokens xuất hiện "toàn phần"
def set_shared_words_whole(row_data):
    return str_common_tokens(row_data[0], row_data[1])

In [ ]:
# Tính tổng tất cả các Từ xuất hiện "một phần"
def set_shared_words_part(row_data):
    return str_common_word(row_data[0], row_data[1])

# **Extract features ở tập train**

In [ ]:
# tính độ dài của search_term
dt_train['len_of_querry'] = [len(_.split()) for _ in dt_train['search_term'].values]

In [ ]:
# tính tổng số lần các tokens ở trường search_term xuất hiện toàn phần ở trường product_title và product_description_attributes
dt_train['shared_words_whole_st_pt'] = [set_shared_words_whole(_) for _ in  dt_train[['search_term','product_title']].values]
dt_train['shared_words_whole_st_pdat'] = [set_shared_words_whole(_) for _ in  dt_train[['search_term','product_description_attributes']].values]

In [ ]:
# tính tổng số lần các tokens ở trường search_term xuất hiện một phần ở trường product_title và product_description_attributes
dt_train['shared_words_part_st_pt'] = [set_shared_words_part(_) for _ in dt_train[['search_term', 'product_title']].values]
dt_train['shared_words_part_st_pdat'] = [set_shared_words_part(_) for _ in dt_train[['search_term', 'product_description_attributes']].values]

In [ ]:
# tính độ tương đồng của search_term và product_title
dt_train['similarity'] = [Levenshtein.ratio(_[0], _[1]) for _ in dt_train[['search_term', 'product_title']].values]

In [ ]:
#show kết quả
dt_train.head(5)

# **Extract features ở tập test**

In [ ]:
# tính độ dài của search_term
dt_test['len_of_querry'] = [len(_.split()) for _ in dt_test['search_term'].values]

In [ ]:
# tính tổng số lần các tokens ở trường search_term xuất hiện toàn phần ở trường product_title và product_description_attributes
dt_test['shared_words_whole_st_pt'] = [set_shared_words_whole(_) for _ in dt_test[['search_term', 'product_title']].values]
dt_test['shared_words_whole_st_pdat'] = [set_shared_words_whole(_) for _ in dt_test[['search_term', 'product_description_attributes']].values]

In [ ]:
# tính tổng số lần các tokens ở trường search_term xuất hiện một phần ở trường product_title và product_description_attributes
dt_test['shared_words_part_st_pt'] = [set_shared_words_part(_) for _ in dt_test[['search_term', 'product_title']].values]
dt_test['shared_words_part_st_pdat'] = [set_shared_words_part(_) for _ in dt_test[['search_term', 'product_description_attributes']].values]

In [ ]:
# tính độ tương đồng của search_term và product_title
dt_test['similarity'] = [Levenshtein.ratio(_[0], _[1]) for _ in dt_test[['search_term', 'product_title']].values]

In [ ]:
#show kết quả
dt_test.head(5)

**Đánh giá lại chất lượng của các features bằng hàm correlation sử dụng tập dt_train**

In [ ]:
correlation(dt_train, 'len_of_querry', transform=False)

In [ ]:
correlation(dt_train, 'shared_words_whole_st_pt', transform=False)

In [ ]:
correlation(dt_train, 'shared_words_whole_st_pdat', transform=False)

In [ ]:
correlation(dt_train, 'shared_words_part_st_pt', transform=False)

In [ ]:
correlation(dt_train, 'shared_words_part_st_pdat', transform=False)

In [ ]:
correlation(dt_train, 'similarity', transform=False)

In [ ]:
# gỡ feature shared_words_part_st_pdat ở 2 tập train và test
dt_train = dt_train.drop(['shared_words_part_st_pdat'],axis=1)
dt_test = dt_test.drop(['shared_words_part_st_pdat'],axis=1)

# **Tách dữ liệu**

In [ ]:
# drop các cột text chỉ để lại các cột chứa features
dt_train = dt_train.drop(['product_title','search_term','product_description_attributes'],axis=1)

# set x_train và y_train
y_train = dt_train['relevance'].values
X_train = dt_train.drop(['id','relevance'], axis=1).values

In [ ]:
#show kết quả
dt_train.head(5)

In [ ]:
# ở tập test cũng gỡ hết các trường khác chỉ để lại các trường mang features
X_test = dt_test.drop(['id','product_title','search_term','product_description_attributes'],axis=1).values

# id test ở đây chính là id của các cặp search_term và product title
id_test = dt_test['id']

# **Random forest regressor**

* Tiến hành train tập dữ liệu với mô hình "Random forest" được cung cấp trong thư viện sklearn

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# bộ tham số được tham khảo từ nguồn khác
rfr = RandomForestRegressor(n_estimators=30, n_jobs=-1, random_state=17, verbose=1.0, max_depth=10)

# fit
rfr.fit(X_train, y_train)

# dự đoán với mô hình vừa fit
y_pred = rfr.predict(X_test)

# xuất file csv
pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission_ntko.csv',index=False)